In [ ]:
import pandas as pd
data=pd.read_csv('/content/IMDB_cleaned.csv',index_col=None)
data.head()

,stopwords_removed,sentiment
0,dead hero get stupid line like woefully abysma...,0
1,think maybe maybe could good early appearance ...,0
2,elite american military team course happen inc...,0
3,ridiculous horror film wealthy man john carrad...,0
4,well one katanas filmnut like sure appreciate ...,1


In [ ]:
corpus=data['stopwords_removed']
lst_corpus = []
for string in corpus:
   lst_words = string.split()
   lst_grams = [" ".join(lst_words[i:i+1]) for i in range(0, len(lst_words), 1)]
   lst_corpus.append(lst_grams)

In [ ]:
from tensorflow.keras import models, layers, preprocessing as kprocessing
tokenizer = kprocessing.text.Tokenizer(lower=True, split='',oov_token="NaN",)
tokenizer.fit_on_texts(lst_corpus)
dic_vocabulary = tokenizer.word_index
lst_sequences= tokenizer.texts_to_sequences(lst_corpus)

In [ ]:
X_train_seq = kprocessing.sequence.pad_sequences(lst_sequences, maxlen=100, padding="post", truncating="post")

In [ ]:
y=data['sentiment']
#y=y.map({'positive':1,'negative':0})

In [ ]:
y

,sentiment
0,0
1,0
2,0
3,0
4,1
...,...
3995,0
3996,1
3997,0
3998,1


In [ ]:
%%time
import gensim.downloader
#glove_vectors = gensim.downloader.load('word2vec-google-news-300' )
#glove_vectors = gensim.downloader.load('glove-twitter-100' )
glove_vectors = gensim.downloader.load('glove-twitter-200')

[==================================================] 100.0% 758.5/758.5MB downloaded
CPU times: user 7min 15s, sys: 28.1 s, total: 7min 44s
Wall time: 9min 23s


In [ ]:
import numpy as np
embeddings = np.zeros((len(dic_vocabulary)+1, 300))
for word,idx in dic_vocabulary.items():
    try:
        embeddings[idx]=glove_vectors[word]
    except:
        pass

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_seq, y, test_size=0.1, random_state=1000)

In [ ]:
y_train_=pd.get_dummies(y_train)
y_test_=pd.get_dummies(y_test)

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers import  Input
input = Input(shape=(100,))
x= layers.Embedding(input_dim=embeddings.shape[0], output_dim=embeddings.shape[1], weights=[embeddings],trainable=False)(input)

In [ ]:
from tensorflow.keras.layers import SimpleRNN,Dense
l1=SimpleRNN(50,return_sequences=False, activation='tanh')(x)
output=Dense(1,activation='sigmoid')(l1)

In [ ]:
# prompt: apply relu activation instead of sigmoid

#output=Dense(1,activation='relu')(l1)


In [ ]:
model=models.Model(input,output)

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
model.fit(X_train,y_train,batch_size=100, epochs=30,validation_split=0.1)

Epoch 1/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 8s 177ms/step - accuracy: 0.4848 - loss: 0.6944 - val_accuracy: 0.4722 - val_loss: 0.6932
Epoch 2/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 8s 111ms/step - accuracy: 0.5051 - loss: 0.6934 - val_accuracy: 0.5278 - val_loss: 0.6928
Epoch 3/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 10s 261ms/step - accuracy: 0.4935 - loss: 0.6933 - val_accuracy: 0.4722 - val_loss: 0.6933
Epoch 4/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 5s 96ms/step - accuracy: 0.4965 - loss: 0.6932 - val_accuracy: 0.4722 - val_loss: 0.6936
Epoch 5/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 92ms/step - accuracy: 0.5101 - loss: 0.6931 - val_accuracy: 0.4722 - val_loss: 0.6932
Epoch 6/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 96ms/step - accuracy: 0.5011 - loss: 0.6932 - val_accuracy: 0.4722 - val_loss: 0.6932
Epoch 7/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 6s 121ms/step - accuracy: 0.4928 - loss: 0.6932 - val_accuracy: 0.4722 - val_loss: 0.6933
Epoch 8/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 93ms/step - accuracy: 0.5017 - loss: 0.6932 - val_accuracy: 0.472

In [ ]:
res=model.predict(X_test)
rounded = [round(x[0]) for x in res]


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step


In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,rounded)
print(cm)

[[  0 206]
 [  0 194]]


In [ ]:
from sklearn.metrics import classification_report
# Print the classification report
print(classification_report(y_test, rounded))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       206
           1       0.48      1.00      0.65       194

    accuracy                           0.48       400
   macro avg       0.24      0.50      0.33       400
weighted avg       0.24      0.48      0.32       400



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
